# CLASS: LB01


### Bryan Leonardo				(2201745004)
### Daniel Richardo 					(2201744531)
### Derren Delano Soerjadi				 (2201740571)
### Giodio Nathanael Pratama Mitaart		 (2201768310)
### John Andray William Doney			 (2201812570)

In [182]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [183]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import metrics

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Getting the dataset

In [184]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IR Project/Anime.csv')

In [185]:
print(dataset.head())
for col in dataset.columns: 
    print(col) 

   animeID  ...                                            related
0        1  ...  {'Adaptation': [{'mal_id': 173, 'type': 'manga...
1        5  ...  {'Parent story': [{'mal_id': 1, 'type': 'anime...
2        6  ...  {'Adaptation': [{'mal_id': 703, 'type': 'manga...
3        7  ...                                                 {}
4        8  ...  {'Adaptation': [{'mal_id': 1348, 'type': 'mang...

[5 rows x 27 columns]
animeID
 name
 title_english
 title_japanese
 title_synonyms
 type
 source
 producers
 genre
 studio
 episodes
 status
 airing
 aired
 duration
 rating
 score
 scored_by
 rank
 popularity
 members
 favorites
 synopsis
 background
 premiered
 broadcast
 related


In [186]:
data = pd.DataFrame()
data[['name', 'synopsis', 'genres']] = dataset[[' name',  " synopsis", " genre"]].copy()
data

,name,synopsis,genres
0,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
1,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...","['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']"
2,Trigun,"Vash the Stampede is the man with a $$60,000,0...","['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D..."
3,Witch Hunter Robin,Witches are individuals with special powers li...,"['Action', 'Magic', 'Police', 'Supernatural', ..."
4,Bouken Ou Beet,It is the dark century and the people are suff...,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu..."
...,...,...,...
15273,Beastars,In a world populated by anthropomorphic animal...,"['Slice of Life', 'Psychological', 'Drama', 'S..."
15274,Mairimashita! Iruma-kun,Suzuki Iruma has been sold to the demon by his...,"['Comedy', 'Demons', 'Supernatural', 'Fantasy'..."
15275,Karrimor Mountain Club,A series of animated commercials produced by K...,"['Adventure', 'Slice of Life']"
15276,Kanata no Astra,"The year is 2061, when space travel is now pos...","['Action', 'Sci-Fi', 'Shounen', 'Space']"


# Preprocessing the data

## Preprocessing the texts

In [187]:
data.isna().values.any()

True

In [188]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data

,name,synopsis,genres
0,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","['Action', 'Adventure', 'Comedy', 'Drama', 'Sc..."
1,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...","['Action', 'Drama', 'Mystery', 'Sci-Fi', 'Space']"
2,Trigun,"Vash the Stampede is the man with a $$60,000,0...","['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D..."
3,Witch Hunter Robin,Witches are individuals with special powers li...,"['Action', 'Magic', 'Police', 'Supernatural', ..."
4,Bouken Ou Beet,It is the dark century and the people are suff...,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu..."
...,...,...,...
14560,Beastars,In a world populated by anthropomorphic animal...,"['Slice of Life', 'Psychological', 'Drama', 'S..."
14561,Mairimashita! Iruma-kun,Suzuki Iruma has been sold to the demon by his...,"['Comedy', 'Demons', 'Supernatural', 'Fantasy'..."
14562,Karrimor Mountain Club,A series of animated commercials produced by K...,"['Adventure', 'Slice of Life']"
14563,Kanata no Astra,"The year is 2061, when space travel is now pos...","['Action', 'Sci-Fi', 'Shounen', 'Space']"


In [189]:
data.synopsis = data.synopsis.str.lower()
data.genres = data.genres.str.lower()
data.head()

,name,synopsis,genres
0,Cowboy Bebop,"in the year 2071, humanity has colonized sever...","['action', 'adventure', 'comedy', 'drama', 'sc..."
1,Cowboy Bebop: Tengoku no Tobira,"another day, another bounty—such is the life o...","['action', 'drama', 'mystery', 'sci-fi', 'space']"
2,Trigun,"vash the stampede is the man with a $$60,000,0...","['action', 'sci-fi', 'adventure', 'comedy', 'd..."
3,Witch Hunter Robin,witches are individuals with special powers li...,"['action', 'magic', 'police', 'supernatural', ..."
4,Bouken Ou Beet,it is the dark century and the people are suff...,"['adventure', 'fantasy', 'shounen', 'supernatu..."


In [190]:
for i in range(data.shape[0]):
  data.synopsis[i] = re.sub(r'\d+','',data.synopsis[i])
data.head()

,name,synopsis,genres
0,Cowboy Bebop,"in the year , humanity has colonized several o...","['action', 'adventure', 'comedy', 'drama', 'sc..."
1,Cowboy Bebop: Tengoku no Tobira,"another day, another bounty—such is the life o...","['action', 'drama', 'mystery', 'sci-fi', 'space']"
2,Trigun,"vash the stampede is the man with a $$,,, boun...","['action', 'sci-fi', 'adventure', 'comedy', 'd..."
3,Witch Hunter Robin,witches are individuals with special powers li...,"['action', 'magic', 'police', 'supernatural', ..."
4,Bouken Ou Beet,it is the dark century and the people are suff...,"['adventure', 'fantasy', 'shounen', 'supernatu..."


In [191]:
for i in range(data.shape[0]):
  data.synopsis[i] = data.synopsis[i].translate(str.maketrans("","",string.punctuation))
data.head()

,name,synopsis,genres
0,Cowboy Bebop,in the year humanity has colonized several of...,"['action', 'adventure', 'comedy', 'drama', 'sc..."
1,Cowboy Bebop: Tengoku no Tobira,another day another bounty—such is the life of...,"['action', 'drama', 'mystery', 'sci-fi', 'space']"
2,Trigun,vash the stampede is the man with a bounty on...,"['action', 'sci-fi', 'adventure', 'comedy', 'd..."
3,Witch Hunter Robin,witches are individuals with special powers li...,"['action', 'magic', 'police', 'supernatural', ..."
4,Bouken Ou Beet,it is the dark century and the people are suff...,"['adventure', 'fantasy', 'shounen', 'supernatu..."


In [192]:
data.synopsis = data.synopsis.str.replace('([^\sA-Za-z])', ' ')
data.head()

,name,synopsis,genres
0,Cowboy Bebop,in the year humanity has colonized several of...,"['action', 'adventure', 'comedy', 'drama', 'sc..."
1,Cowboy Bebop: Tengoku no Tobira,another day another bounty such is the life of...,"['action', 'drama', 'mystery', 'sci-fi', 'space']"
2,Trigun,vash the stampede is the man with a bounty on...,"['action', 'sci-fi', 'adventure', 'comedy', 'd..."
3,Witch Hunter Robin,witches are individuals with special powers li...,"['action', 'magic', 'police', 'supernatural', ..."
4,Bouken Ou Beet,it is the dark century and the people are suff...,"['adventure', 'fantasy', 'shounen', 'supernatu..."


In [193]:
for i in range(data.shape[0]):
  data.synopsis[i] = data.synopsis[i].strip()
data.head()

,name,synopsis,genres
0,Cowboy Bebop,in the year humanity has colonized several of...,"['action', 'adventure', 'comedy', 'drama', 'sc..."
1,Cowboy Bebop: Tengoku no Tobira,another day another bounty such is the life of...,"['action', 'drama', 'mystery', 'sci-fi', 'space']"
2,Trigun,vash the stampede is the man with a bounty on...,"['action', 'sci-fi', 'adventure', 'comedy', 'd..."
3,Witch Hunter Robin,witches are individuals with special powers li...,"['action', 'magic', 'police', 'supernatural', ..."
4,Bouken Ou Beet,it is the dark century and the people are suff...,"['adventure', 'fantasy', 'shounen', 'supernatu..."


In [194]:
stop_words = set(stopwords.words('english'))
for i in range(data.shape[0]):
  synopsis_token = word_tokenize(str(data.synopsis[i]))
  data.synopsis[i] = [word for word in synopsis_token if not word in stop_words]
data.head()

,name,synopsis,genres
0,Cowboy Bebop,"[year, humanity, colonized, several, planets, ...","['action', 'adventure', 'comedy', 'drama', 'sc..."
1,Cowboy Bebop: Tengoku no Tobira,"[another, day, another, bounty, life, often, u...","['action', 'drama', 'mystery', 'sci-fi', 'space']"
2,Trigun,"[vash, stampede, man, bounty, head, reason, he...","['action', 'sci-fi', 'adventure', 'comedy', 'd..."
3,Witch Hunter Robin,"[witches, individuals, special, powers, like, ...","['action', 'magic', 'police', 'supernatural', ..."
4,Bouken Ou Beet,"[dark, century, people, suffering, rule, devil...","['adventure', 'fantasy', 'shounen', 'supernatu..."


In [195]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stem_list(row):
    my_list = row['synopsis']
    stemmed_list = [stemmer.stem(word) for word in my_list]
    return (stemmed_list)

data['stemmed_words'] = data.apply(stem_list, axis=1)
data["stemmed_words"]

0        [year, human, colon, sever, planet, moon, sola...
1        [anoth, day, anoth, bounti, life, often, unluc...
2        [vash, stamped, man, bounti, head, reason, he,...
3        [witch, individu, special, power, like, esp, t...
4        [dark, centuri, peopl, suffer, rule, devil, va...
                               ...                        
14560    [world, popul, anthropomorph, anim, herbivor, ...
14561    [suzuki, iruma, sold, demon, irrespons, parent...
14562    [seri, anim, commerci, produc, karrimor, japan...
14563    [year, space, travel, possibl, commerci, viabl...
14564    [civil, war, north, desper, defeat, south, uti...
Name: stemmed_words, Length: 14565, dtype: object

In [196]:
def join(row):
  separator = ' '
  return separator.join(row['stemmed_words'])

data['join_synopsis'] = data.apply(join, axis=1)
data["join_synopsis"]

0        year human colon sever planet moon solar syste...
1        anoth day anoth bounti life often unlucki crew...
2        vash stamped man bounti head reason he mercile...
3        witch individu special power like esp telekine...
4        dark centuri peopl suffer rule devil vandel ab...
                               ...                        
14560    world popul anthropomorph anim herbivor carniv...
14561    suzuki iruma sold demon irrespons parent excha...
14562    seri anim commerci produc karrimor japan promo...
14563    year space travel possibl commerci viabl stude...
14564    civil war north desper defeat south util forbi...
Name: join_synopsis, Length: 14565, dtype: object

## Preprocessing the target

In [197]:
data.genres

0        ['action', 'adventure', 'comedy', 'drama', 'sc...
1        ['action', 'drama', 'mystery', 'sci-fi', 'space']
2        ['action', 'sci-fi', 'adventure', 'comedy', 'd...
3        ['action', 'magic', 'police', 'supernatural', ...
4        ['adventure', 'fantasy', 'shounen', 'supernatu...
                               ...                        
14560    ['slice of life', 'psychological', 'drama', 's...
14561    ['comedy', 'demons', 'supernatural', 'fantasy'...
14562                       ['adventure', 'slice of life']
14563             ['action', 'sci-fi', 'shounen', 'space']
14564                               ['fantasy', 'shounen']
Name: genres, Length: 14565, dtype: object

In [198]:
temp = pd.DataFrame(data.genres) 
temp

,genres
0,"['action', 'adventure', 'comedy', 'drama', 'sc..."
1,"['action', 'drama', 'mystery', 'sci-fi', 'space']"
2,"['action', 'sci-fi', 'adventure', 'comedy', 'd..."
3,"['action', 'magic', 'police', 'supernatural', ..."
4,"['adventure', 'fantasy', 'shounen', 'supernatu..."
...,...
14560,"['slice of life', 'psychological', 'drama', 's..."
14561,"['comedy', 'demons', 'supernatural', 'fantasy'..."
14562,"['adventure', 'slice of life']"
14563,"['action', 'sci-fi', 'shounen', 'space']"


In [199]:
ast.literal_eval(data['genres'].iloc[0])

['action', 'adventure', 'comedy', 'drama', 'sci-fi', 'space']

In [200]:
data['genres'] = data['genres'].apply(lambda x: ast.literal_eval(x))
data['genres'].head()

0    [action, adventure, comedy, drama, sci-fi, space]
1              [action, drama, mystery, sci-fi, space]
2    [action, sci-fi, adventure, comedy, drama, sho...
3    [action, magic, police, supernatural, drama, m...
4          [adventure, fantasy, shounen, supernatural]
Name: genres, dtype: object

In [201]:

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data['genres'])

# transform target variable
y = multilabel_binarizer.transform(data.genres)

In [202]:
multilabel_binarizer.classes_

array(['action', 'adventure', 'cars', 'comedy', 'dementia', 'demons',
       'drama', 'ecchi', 'fantasy', 'game', 'harem', 'hentai',
       'historical', 'horror', 'josei', 'kids', 'magic', 'martial arts',
       'mecha', 'military', 'music', 'mystery', 'parody', 'police',
       'psychological', 'romance', 'samurai', 'school', 'sci-fi',
       'seinen', 'shoujo', 'shoujo ai', 'shounen', 'shounen ai',
       'slice of life', 'space', 'sports', 'super power', 'supernatural',
       'thriller', 'vampire', 'yaoi', 'yuri'], dtype=object)

In [203]:
y

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [204]:
temp

,genres
0,"['action', 'adventure', 'comedy', 'drama', 'sc..."
1,"['action', 'drama', 'mystery', 'sci-fi', 'space']"
2,"['action', 'sci-fi', 'adventure', 'comedy', 'd..."
3,"['action', 'magic', 'police', 'supernatural', ..."
4,"['adventure', 'fantasy', 'shounen', 'supernatu..."
...,...
14560,"['slice of life', 'psychological', 'drama', 's..."
14561,"['comedy', 'demons', 'supernatural', 'fantasy'..."
14562,"['adventure', 'slice of life']"
14563,"['action', 'sci-fi', 'shounen', 'space']"


In [205]:
for i in range(data.shape[0]):
  temp.genres[i] = temp.genres[i].translate(str.maketrans("","",string.punctuation))
unique = set(temp.genres.str.split(' ').sum())

In [206]:
test = np.zeros(shape=(14565,))

In [207]:
target = pd.DataFrame(test)
for x in unique:
  if x != '':
    target[x] = 0
target.shape

(14565, 47)

In [208]:
target.drop(target.columns[0], axis=1, inplace=True)
target.drop('life', axis=1, inplace=True)
target.drop('of', axis=1, inplace=True)
target.shape

(14565, 44)

In [209]:
i = 0
for x in temp.genres:
  unique = x.split()
  for z in unique:
    if z == 'of' or z == 'life':
      continue
    target[z][i] = 1
    
  i += 1
target

,ai,horror,romance,comedy,fantasy,super,harem,action,scifi,mecha,mystery,music,drama,samurai,hentai,vampire,parody,shounen,psychological,police,seinen,school,historical,arts,sports,yaoi,dementia,adventure,space,slice,martial,game,supernatural,demons,shoujo,thriller,yuri,cars,magic,military,ecchi,kids,power,josei
0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14560,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14561,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
14562,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14563,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [210]:
data['list'] = target[target.columns[0:]].values.tolist()
new_df = data[['join_synopsis', 'list']].copy()
new_df.head()

,join_synopsis,list
0,year human colon sever planet moon solar syste...,"[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, ..."
1,anoth day anoth bounti life often unlucki crew...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, ..."
2,vash stamped man bounti head reason he mercile...,"[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, ..."
3,witch individu special power like esp telekine...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, ..."
4,dark centuri peopl suffer rule devil vandel ab...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Normalize

In [211]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [212]:
# split dataset into training and validation set
xtrain, xval, ytrain, yval = train_test_split(data["join_synopsis"], y, test_size=0.2, random_state=9)

## TF-IDF

In [213]:
# create TF-IDF features

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

# Build the model

In [214]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

## Logistic linear

In [215]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [216]:
# fit model on train data
clf.fit(xtrain_tfidf, ytrain)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [217]:
# make predictions for validation set
y_pred = clf.predict(xval_tfidf)

In [218]:
y_pred[3]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [219]:
multilabel_binarizer.inverse_transform(y_pred)[3]

('comedy',)

In [220]:
# evaluate performance
f1_score(yval, y_pred, average="micro")

0.33521870286576166

In [221]:
multilabel_binarizer.classes_

array(['action', 'adventure', 'cars', 'comedy', 'dementia', 'demons',
       'drama', 'ecchi', 'fantasy', 'game', 'harem', 'hentai',
       'historical', 'horror', 'josei', 'kids', 'magic', 'martial arts',
       'mecha', 'military', 'music', 'mystery', 'parody', 'police',
       'psychological', 'romance', 'samurai', 'school', 'sci-fi',
       'seinen', 'shoujo', 'shoujo ai', 'shounen', 'shounen ai',
       'slice of life', 'space', 'sports', 'super power', 'supernatural',
       'thriller', 'vampire', 'yaoi', 'yuri'], dtype=object)

In [222]:
from sklearn import metrics
print(metrics.classification_report(yval, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.33      0.44       635
           1       0.78      0.25      0.38       475
           2       1.00      0.07      0.13        29
           3       0.73      0.48      0.58      1049
           4       0.94      0.21      0.35        70
           5       0.67      0.06      0.11        64
           6       0.60      0.07      0.12       435
           7       0.00      0.00      0.00       141
           8       0.78      0.24      0.36       499
           9       0.50      0.02      0.03        58
          10       0.00      0.00      0.00        73
          11       0.96      0.26      0.41       257
          12       0.91      0.06      0.10       180
          13       0.00      0.00      0.00        77
          14       0.00      0.00      0.00        18
          15       0.84      0.30      0.44       398
          16       0.64      0.05      0.10       171
          17       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [223]:
# predict probabilities
y_pred_prob = clf.predict_proba(xval_tfidf)

In [224]:
t = 0.25 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

# evaluate performance
f1_score(yval, y_pred_new, average="micro")

0.520544443734424

In [225]:
# function for text cleaning 
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text
# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

In [226]:
def infer_tags(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

In [227]:
for i in range(5): 
  k = xval.sample(1).index[0] 
  print("Anime: ", data['name'][k], 
        "\nPredicted genre: ", infer_tags(xval[k])), print("Actual genre: ",data['genres'][k], "\n")

Anime:  Panda-Z: The Robonimation 
Predicted genre:  [('action', 'comedy', 'mecha')]
Actual genre:  ['action', 'comedy', 'kids', 'mecha', 'parody', 'sci-fi', 'shounen'] 

Anime:  Kurogane Communication 
Predicted genre:  [('sci-fi',)]
Actual genre:  ['action', 'adventure', 'drama', 'sci-fi'] 

Anime:  Soukou Kihei Votoms: Vol.1 Stories of the 'A.T. Votoms' 
Predicted genre:  [()]
Actual genre:  ['action', 'mecha', 'military', 'drama', 'sci-fi'] 

Anime:  Shimajirou no Wow! 
Predicted genre:  [('comedy', 'kids')]
Actual genre:  ['adventure', 'comedy', 'fantasy', 'kids', 'magic'] 

Anime:  Ishindenshin Shiyou 
Predicted genre:  [('music',)]
Actual genre:  ['music', 'kids'] 



## Deep Learning

In [228]:
!pip install transformers

In [229]:
# Importing stock ml libraries
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [230]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 4
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [231]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.join_synopsis = dataframe.join_synopsis
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.join_synopsis)

    def __getitem__(self, index):
        join_synopsis = str(self.join_synopsis[index])
        join_synopsis = " ".join(join_synopsis.split())

        inputs = self.tokenizer.encode_plus(
            join_synopsis,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        # ids = torch.tensor(inputs['input_ids'].to_numpy())
        # mask = torch.tensor(inputs['attention_mask'].to_numpy())
        # token_type_ids = torch.tensor(inputs["token_type_ids"].to_numpy())

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [232]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset =new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (14565, 2)
TRAIN Dataset: (11652, 2)
TEST Dataset: (2913, 2)


In [233]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [234]:
target.shape

(14565, 44)

In [235]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 44)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [236]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [237]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [238]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [239]:
for epoch in range(EPOCHS):
    train(epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  0.7096258401870728
Epoch: 1, Loss:  0.22557686269283295
Epoch: 2, Loss:  0.13724632561206818
Epoch: 3, Loss:  0.2148236632347107


In [240]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [241]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Accuracy Score = 0.1283899759697906
F1 Score (Micro) = 0.4093576785132051
F1 Score (Macro) = 0.16810696940340908
Accuracy Score = 0.1283899759697906
F1 Score (Micro) = 0.4093576785132051
F1 Score (Macro) = 0.16810696940340908
Accuracy Score = 0.1283899759697906
F1 Score (Micro) = 0.4093576785132051
F1 Score (Macro) = 0.16810696940340908
Accuracy Score = 0.1283899759697906
F1 Score (Micro) = 0.4093576785132051
F1 Score (Macro) = 0.16810696940340908
